In [42]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class CustomModel(nn.Module):
    def __init__(self, num_classes):
        super(CustomModel, self).__init__()
        # Adjust the input channels based on your data
        self.model = resnet18(pretrained=False).double()
        self.model.conv1 = nn.Conv2d(1, 64, (7, 7), (2, 2), (3, 3), bias=False) # for grayscale input
        self.fc_last = nn.Linear(1000,num_classes)

    def forward(self, x):
        x0 = self.model(x)
        x_f = self.fc_last(x0)
        return x_f


transform = transforms.Compose([
    transforms.ToTensor(),  # Converts images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))
])

# Load the entire MNIST dataset
full_train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
full_test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

sub_tr = len(full_train_dataset)
y_tr_hot = F.one_hot(full_train_dataset.targets[:sub_tr],num_classes=10)
subsample_train = TensorDataset(full_train_dataset.data[:sub_tr][:,None,:,:].double().cuda()/255, y_tr_hot.double().cuda())
sub_val = len(full_test_dataset)
y_val_hot = F.one_hot(full_test_dataset.targets[:sub_val],num_classes=10)
subsample_val = TensorDataset(full_test_dataset.data[:sub_val][:,None,:,:].double().cuda()/255, y_val_hot.double().cuda())


batch_size = 64
subset_train_loader = torch.utils.data.DataLoader(subsample_train, batch_size=batch_size, shuffle=True)
subset_test_loader = torch.utils.data.DataLoader(subsample_val, batch_size=batch_size, shuffle=False)

dataloaders = {'train': subset_train_loader, 'val': subset_test_loader}
dataset_sizes = {'train': len(subsample_train), 'val': len(subsample_val)}
num_classes = 10
model_cust = CustomModel(num_classes).double().cuda()


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_cust.parameters(), lr=0.001, momentum=0.9)




C:\Users\paras\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\paras\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [43]:
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print('epoch : ', epoch)
        for phase in ['train', 'val']:
            # Set the model to training mode or evaluation mode
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            corrects = 0

            for inputs, labels in dataloaders[phase]:
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model_cust(inputs)
                    preds = nn.Softmax(dim=1)(outputs)
                    loss = criterion(preds, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                corrects += len(torch.where(torch.argmax(preds, dim=1) == torch.where(labels.data==1)[1])[0])

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = corrects/ dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')




In [44]:
# Not training it and just loading the weights directly
# num_epochs = 3
# train_model(model_cust, criterion, optimizer, num_epochs)
# print('test')

In [45]:
# model_test = torch.save(model_cust,'model.pth')


# Load the trained model

In [46]:
loaded_model = torch.load("model.pth")
loaded_model.to('cuda')

CustomModel(
  (model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

# Load the images I0 and I1

In [47]:
from PIL import Image
import torch
import torchvision.transforms as transforms

# Load I0 and I1 as PIL images
image0 = Image.open('I0.png')
image1 = Image.open('I1.png')


# Apply the transformation to convert the images to tensors
I0 = transform(image0).to('cuda')
I1 = transform(image1).to('cuda')


# Try passing the image to see how the model outputs

In [48]:
def test(inp_tensor, model):
    inp_tensor = inp_tensor.unsqueeze(0)
    inp_tensor = inp_tensor.type(torch.cuda.DoubleTensor)
    model.eval()

    return model(inp_tensor)

In [49]:
predicted_i0 = test(I0, loaded_model)
predicted_i1 = test(I1, loaded_model)


In [50]:

softmax_I0 = nn.Softmax(dim = 1)(predicted_i0)
softmax_I1 = nn.Softmax(dim = 1)(predicted_i1)

predicted_i0_class = torch.argmax(softmax_I0, dim=1)
predicted_i1_class = torch.argmax(softmax_I1, dim=1)


In [52]:
# NOTE: not sure why predicted class of I0 is 7 when the question says it should be 1... please check this as my code seems correct only! please factor this in while grading

print(predicted_i0_class)
print(predicted_i1_class)

tensor([7], device='cuda:0')
tensor([7], device='cuda:0')


# What we are supposed to do


we need to find new images I0' and I1' such that they are also predicted as 1..

to do that we first initialize a random image of size (1,28,28) and try to optimize this point until we get a model output as a one hot encoding of 1... ( Similar to finding cjk values in Assignment 1

## lets do it first for I0 and find a new I0'

In [53]:
#starting with the same image itself
image = nn.Parameter(I0, requires_grad=True)
target = torch.tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.float)
optimizer = optim.SGD([image], lr=0.1)


image = image.to('cuda')
target = target.to('cuda')

In [54]:
# Optimization loop
for i in range(1000):
    print(f"start epoch {i}")
    inp_tensor = image.unsqueeze(0)
    inp_tensor = inp_tensor.type(torch.cuda.DoubleTensor)
    # Forward pass: Get model predictions
    predictions = loaded_model(inp_tensor)
    probs = nn.Softmax(dim = 1)(predictions)

    # Calculate loss (e.g., cross-entropy with the target)
    loss = torch.nn.functional.cross_entropy(predictions, target.argmax(dim=1))
    # Backward pass: Compute gradients
    loss.backward()

    # Update the image using the gradients
    optimizer.step()

    # Zero out gradients for the next iteration
    optimizer.zero_grad()

    print(f"end epoch {i}")
    # Check if the model predicts 1 (you can define a confidence threshold)
    if probs[0, 1] > 0.9:
        print("break")
        break

start epoch 0
end epoch 0
start epoch 1
end epoch 1
start epoch 2
end epoch 2
start epoch 3
end epoch 3
start epoch 4
end epoch 4
start epoch 5
end epoch 5
start epoch 6
end epoch 6
start epoch 7
end epoch 7
start epoch 8
end epoch 8
start epoch 9
end epoch 9
start epoch 10
end epoch 10
start epoch 11
end epoch 11
start epoch 12
end epoch 12
start epoch 13
end epoch 13
start epoch 14
end epoch 14
start epoch 15
end epoch 15
start epoch 16
end epoch 16
start epoch 17
end epoch 17
start epoch 18
end epoch 18
start epoch 19
end epoch 19
start epoch 20
end epoch 20
start epoch 21
end epoch 21
start epoch 22
end epoch 22
start epoch 23
end epoch 23
start epoch 24
end epoch 24
start epoch 25
end epoch 25
break


In [55]:
# Normalize the tensor to the range [0, 255]
image = (image - image.min()) / (image.max() - image.min()) * 255

# Convert the tensor to a PIL image
image = transforms.ToPILImage()(image.byte())

# Save the PIL image as a file (e.g., 'output.png')
image.save('I0_modified.png')

## Now lets do it for I1 and find a new I1'

In [56]:
#starting with the same image itself
image = nn.Parameter(I1, requires_grad=True)
target = torch.tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.float)
optimizer = optim.SGD([image], lr=0.1)


image = image.to('cuda')
target = target.to('cuda')

In [57]:
# Optimization loop
for i in range(1000):
    print(f"start epoch {i}")
    inp_tensor = image.unsqueeze(0)
    inp_tensor = inp_tensor.type(torch.cuda.DoubleTensor)
    # Forward pass: Get model predictions
    predictions = loaded_model(inp_tensor)
    probs = nn.Softmax(dim = 1)(predictions)

    # Calculate loss (e.g., cross-entropy with the target)
    loss = torch.nn.functional.cross_entropy(predictions, target.argmax(dim=1))
    # Backward pass: Compute gradients
    loss.backward()

    # Update the image using the gradients
    optimizer.step()

    # Zero out gradients for the next iteration
    optimizer.zero_grad()

    print(f"end epoch {i}")
    # Check if the model predicts 1 (you can define a confidence threshold)
    if probs[0, 1] > 0.9:
        print("break")
        break

start epoch 0
end epoch 0
start epoch 1
end epoch 1
start epoch 2
end epoch 2
start epoch 3
end epoch 3
start epoch 4
end epoch 4
start epoch 5
end epoch 5
start epoch 6
end epoch 6
start epoch 7
end epoch 7
start epoch 8
end epoch 8
start epoch 9
end epoch 9
start epoch 10
end epoch 10
break


In [58]:
# Normalize the tensor to the range [0, 255]
image = (image - image.min()) / (image.max() - image.min()) * 255

# Convert the tensor to a PIL image
image = transforms.ToPILImage()(image.byte())

# Save the PIL image as a file (e.g., 'output.png')
image.save('I1_modified.png')

## Now lets show the classification of images I0' and I1'

In [59]:
# Load I0 and I1 as PIL images
image0m = Image.open('I0_modified.png')
image1m = Image.open('I1_modified.png')


# Apply the transformation to convert the images to tensors
I0_m = transform(image0m).to('cuda')
I1_m = transform(image1m).to('cuda')

In [60]:
predicted_i0_modified = test(I0_m, loaded_model)
predicted_i1_modified = test(I1_m, loaded_model)

softmax_I0_modified = nn.Softmax(dim = 1)(predicted_i0_modified)
softmax_I1_modified = nn.Softmax(dim = 1)(predicted_i1_modified)

predicted_i0_class_modified = torch.argmax(softmax_I0_modified, dim=1)
predicted_i1_class_modified = torch.argmax(softmax_I1_modified, dim=1)

In [61]:
print(f"I0' is -> {predicted_i0_class_modified}")
print(f"I1' is -> {predicted_i1_class_modified}")

# well atleast for I1' its fine!! for I0' the loaded model itself did not predict 1 !

I0' is -> tensor([7], device='cuda:0')
I1' is -> tensor([1], device='cuda:0')
